# Image Enhancement API - Google Colab

This notebook runs a Flask API server with ngrok tunnel for the web UI.

## Steps:
1. Runtime > Change runtime type > T4 GPU
2. Run all cells
3. Copy the ngrok URL (https://xxxx.ngrok-free.app)
4. Use it in your web UI `.env.local`

## Step 1: Check GPU

In [30]:
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
else:
    print(" No GPU! Go to Runtime > Change runtime type > T4 GPU")

PyTorch: 2.8.0+cu126
CUDA: True
GPU: Tesla T4
Memory: 14.74 GB


## Step 2: Clone Repository

In [31]:
!git clone https://github.com/vEEr6057/image_bot.git
%cd image_bot
!pwd
!ls -la
print("\n Repository cloned!")

Cloning into 'image_bot'...
remote: Enumerating objects: 246, done.
remote: Counting objects: 100% (246/246), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 246 (delta 129), reused 175 (delta 59), pack-reused 0 (from 0)
Receiving objects: 100% (246/246), 124.53 KiB | 7.78 MiB/s, done.
Resolving deltas: 100% (129/129), done.
/content/image_bot/image_bot/image_bot/image_bot/image_bot
/content/image_bot/image_bot/image_bot/image_bot/image_bot
total 128
drwxr-xr-x 7 root root 4096 Nov 19 10:04 .
drwxr-xr-x 9 root root 4096 Nov 19 10:03 ..
-rw-r--r-- 1 root root 6701 Nov 19 10:04 api_server_colab.py
-rw-r--r-- 1 root root 3677 Nov 19 10:04 api_server.py
-rw-r--r-- 1 root root 6583 Nov 19 10:04 colab_api_setup.ipynb
-rw-r--r-- 1 root root 3189 Nov 19 10:04 COLAB_GUIDE.md
-rw-r--r-- 1 root root 6833 Nov 19 10:04 colab_setup.ipynb
-rw-r--r-- 1 root root 5561 Nov 19 10:04 CONNECT_BACKEND.md
-rw-r--r-- 1 root root  306 Nov 19 10:04 .env.example
drwxr-xr-x 8 root root 4096

## Step 3: Install Dependencies

**Note**: Package version fixes for Colab compatibility:
- Pillow < 12.0 (torchvision requirement)
- requests==2.32.4 (Colab requirement)

In [32]:
# Install packages with compatible versions
!pip install --upgrade opencv-python-headless>=4.9.0.80
!pip install 'Pillow<12.0,>=8.0'  # Fix: torchvision requires Pillow < 12.0
!pip install requests==2.32.4  # Fix: Colab requires exact version
!pip install flask flask-cors
!pip install pyngrok
print("\n Basic packages installed!")

# Install Real-ESRGAN
print("\n Installing Real-ESRGAN...")
!pip install basicsr facexlib gfpgan realesrgan
print("\n Real-ESRGAN installed!")

# Apply compatibility patch
print("\n Applying compatibility patch...")
import sys
import torchvision.transforms.functional as F

class FunctionalTensorModule:
    @staticmethod
    def rgb_to_grayscale(img, num_output_channels=1):
        return F.rgb_to_grayscale(img, num_output_channels)

sys.modules['torchvision.transforms.functional_tensor'] = FunctionalTensorModule()
print(" Patch applied!")

# Verify imports
print("\n Verifying imports...")
from realesrgan import RealESRGANer
from basicsr.archs.rrdbnet_arch import RRDBNet
print(" All imports successful!")


 Basic packages installed!

 Installing Real-ESRGAN...

 Real-ESRGAN installed!

 Applying compatibility patch...
 Patch applied!

 Verifying imports...
 All imports successful!


## Step 4: Configure Environment

In [33]:
# Create .env file for API server
with open('.env', 'w') as f:
    f.write("USE_GPU=true\n")
    f.write("TILE_SIZE=1024\n")
    f.write("TILE_PAD=64\n")
    f.write("PRE_PAD=10\n")
    f.write("USE_FP16=true\n")

print(" Environment configured!")
print("GPU optimizations: tile=1024, tile_pad=64, FP16=true")

 Environment configured!
GPU optimizations: tile=1024, tile_pad=64, FP16=true


## Step 5: Setup ngrok Tunnel

**Your ngrok token:** `35gxFmvmgMgzqe5Sqij0FNMI6hP_j6BMUX2ioR3wNLj5DW7`

In [34]:
from pyngrok import ngrok

# Set ngrok authtoken
NGROK_AUTH_TOKEN = "35gxFmvmgMgzqe5Sqij0FNMI6hP_j6BMUX2ioR3wNLj5DW7"

ngrok.set_auth_token(NGROK_AUTH_TOKEN)
print(" ngrok configured!")

 ngrok configured!


## Step 6: Start API Server with ngrok

In [ ]:
import threading
import time
from pyngrok import ngrok

# Import standalone Flask app (no complex imports needed)
print(" Loading Flask app...")
from api_server_colab import app
print(" Flask app loaded!")

# Start Flask in background thread
def run_flask():
    app.run(host='0.0.0.0', port=5000, debug=False, use_reloader=False)

print("\n Starting Flask server...")
flask_thread = threading.Thread(target=run_flask, daemon=True)
flask_thread.start()

# Wait for Flask to start
time.sleep(5)
print(" Flask server started!")

# Create ngrok tunnel
print("\n Creating ngrok tunnel...")
public_url = ngrok.connect(5000)

print("\n" + "="*60)
print(" API SERVER IS RUNNING!")
print("="*60)
print(f"\n Public URL: {public_url}")
print("\n Copy this URL and add to web-ui/.env.local:")
print(f"   NEXT_PUBLIC_BACKEND_BASE_URL={public_url}")
print("\n" + "="*60)
print("\n IMPORTANT: Keep this notebook running!")
print("   The API will stop if you close this tab.")
print("\n Test your API:")
print(f"   {public_url}/health")
print("="*60 + "\n")

# Keep running
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("\n Server stopped")
    ngrok.disconnect(public_url)

 Loading Flask app...
 Flask app loaded!

 Starting Flask server...
 * Serving Flask app 'api_server_colab'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


 Flask server started!

 Creating ngrok tunnel...

 API SERVER IS RUNNING!

 Public URL: NgrokTunnel: "https://impleadable-mouthily-julien.ngrok-free.dev" -> "http://localhost:5000"

 Copy this URL and add to web-ui/.env.local:
   NEXT_PUBLIC_BACKEND_BASE_URL=NgrokTunnel: "https://impleadable-mouthily-julien.ngrok-free.dev" -> "http://localhost:5000"


 IMPORTANT: Keep this notebook running!
   The API will stop if you close this tab.

 Test your API:
   NgrokTunnel: "https://impleadable-mouthily-julien.ngrok-free.dev" -> "http://localhost:5000"/health



INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 10:04:49] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 10:04:50] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 10:08:42] "GET / HTTP/1.1" 200 -


	Tile 1/1


INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 10:18:22] "POST /api/upscale HTTP/1.1" 200 -


	Tile 1/1


INFO:werkzeug:127.0.0.1 - - [19/Nov/2025 10:33:01] "POST /api/upscale HTTP/1.1" 200 -
